# DayTime Checkout

This notebook is intended to run a series of tests to ensure the system responds as expected.
It begins with LATISS, but can also be used with the telescope ATCS components only.

In [ ]:
import asyncio
import time
import os
import numpy as np
from astropy.time import Time

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

In [ ]:
logger = logging.getLogger(f"Daytime Checkout {Time.now()} UT")
logger.level = logging.DEBUG

In [ ]:
logger.info(os.environ["OSPL_URI"])
logger.info(os.environ["LSST_DDS_PARTITION_PREFIX"])

Helper function

### Instantiate the control classes

In [ ]:
domain = salobj.Domain()
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

In [ ]:
await latiss.enable()
await atcs.enable()

In [ ]:
await atcs.slew_dome_to(180.0)

In [ ]:
await atcs.open_dome_shutter()

In [ ]:
await atcs.disable_dome_following()

In [ ]:
start_az=180.0
start_el=70.0
start_rot=0
await atcs.point_azel(az=start_az, el=start_el, rot_tel=start_rot)

In [ ]:
coord=atcs.radec_from_azel(az=start_az, el=start_el)
await atcs.slew_icrs(coord.ra, coord.dec, rot=start_rot, stop_before_slew=False, rot_type=RotType.PhysicalSky)

In [ ]:
# Current Gemini wind speed 3.6 m/s from 302 degrees.

### Off

In [ ]:
await latiss.take_darks(30.0, ndarks=5)

### 10Hz

In [ ]:
await latiss.take_darks(30.0, ndarks=5)

### 20 Hz

In [ ]:
await latiss.take_darks(30.0, ndarks=5)

### 30 Hz

In [ ]:
await latiss.take_darks(30.0, ndarks=5)

### 40 Hz

In [ ]:
await latiss.take_darks(30.0, ndarks=5)

### 50 Hz

In [ ]:
await latiss.take_darks(30.0, ndarks=5)

In [ ]:
await atcs.close_dome()

In [ ]:
# Stop tracking
await atcs.stop_tracking()

In [ ]:
start_az=0.0
start_el=80.0
start_rot=0
await atcs.point_azel(az=start_az, el=start_el, rot_tel=start_rot)

## Shutdown all ATCS components

In [ ]:
await atcs.shutdown()

## Put LATISS in standby

In [ ]:
await latiss.standby()